In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import joblib
import math
import numpy as np
import itertools 
from itertools import product
from collections import Counter
import garpar as gp
import re
import skcriteria as skc
from skcriteria import pipeline
from skcriteria.preprocessing import invert_objectives, weighters, scalers
from skcriteria.madm import similarity, moora, electre
from skcriteria.ranksrev import RankInvariantChecker
from garpar.datasets import make_multisector, RissoLevyStable, RissoNormal, RissoUniform

Para el experimento voy a buscar los paràmetros del experimento de EPIO y voy a pedir me de los màs extremos  y un caso medio. Eso se lo pido a chat gtp 
Luego cargo los sectores con el nùmero de acciones de cada sector y los niveles de entropía baja media y alta para todos los sectores
Optimizo y descargo las carteras
Armo el indicador compuesto 
Tenemos que funciona 
Definir nùmero de mercado  simuladas con esos paràmetros=100 con cada paràmetros de sector eficiencia_alta _media_baja para todos los sectores despues combinar 1/1, 1/3 y 1/3 con los niveles de eficiencia 
Sonc 6 sectores* 4 horizontes temporales*3 niveles de entropía* 


In [13]:
por_multisector = gp.datasets.make_multisector(RissoLevyStable(entropy=0.99), RissoLevyStable(entropy=0.01), RissoLevyStable(entropy=0.81), stocks=5, days=25, window_size=3)
por_multisector

Stocks,"rissolevystable_1_S0[W 1.0, H 0.99]","rissolevystable_1_S1[W 1.0, H 0.99]","rissolevystable_2_S0[W 1.0, H 0.01]","rissolevystable_2_S1[W 1.0, H 0.01]","rissolevystable_3_S0[W 1.0, H 0.81]"
Days,,,,,
0,100.000000,100.000000,100.000000,100.000000,100.000000
1,99.998710,99.992121,99.994402,99.982163,99.996372
2,100.006119,99.996122,99.991897,99.982145,99.997334
3,100.014225,99.982604,99.990431,99.980061,99.992689
4,100.024629,99.979507,99.980365,99.975911,99.997157
5,99.996232,99.998017,99.987595,99.966153,100.000557
6,99.996122,99.982463,100.007008,99.961070,99.983325
7,99.999577,99.992441,100.018958,99.963588,99.978490
8,99.989648,99.999005,100.019354,99.959094,99.980455


In [6]:
gp.datasets.make_risso_levy_stable?

Signature:
gp.datasets.make_risso_levy_stable(
    alpha=1.6411,
    beta=-0.0126,
    mu=0.0005,
    sigma=0.005,
    *,
    entropy=0.5,
    random_state=None,
    n_jobs=None,
    verbose=0,
    **kwargs,
)
Docstring:
Create a stocks set using the RissoLevyStable stocks set maker.

Parameters
----------
alpha : float, optional
    Shape parameter of the Levy stable distribution. Default is 1.6411.
beta : float, optional
    Scale parameter of the Levy stable distribution. Default is -0.0126.
mu : float, optional
    Location parameter (mean) of the Levy stable distribution.
    Default is 0.0005.
sigma : float, optional
    Scale parameter (spread) of the Levy stable distribution.
    Default is 0.005.
entropy : float, optional
    Entropy parameter controlling the randomness in stocks set creation.
    Default is 0.5.
random_state : {None, int, numpy.random.Generator}, optional
    Seed or Generator for the random number generator. Default is None.
n_jobs : int, optional
    Number

In [7]:
gp.datasets.make_multisector?

Signature: gp.datasets.make_multisector(*makers, **kwargs)
Docstring:
Create a multi-sector StocksSet using specified sector makers.

Parameters
----------
*makers : variable-length arguments
    Instances of StocksSetMakerABC representing sector makers.
**kwargs : keyword arguments
    Additional parameters passed to MultiSector.make_stocks_set.

Returns
-------
garpar.core.stocks_set.StocksSet
    StocksSet object representing the generated multi-sector stocks set.

Notes
-----
This function creates a multi-sector stocks set by initializing a
MultiSector object with unique names for each sector maker and then
calling make_stocks_set with specified parameters.

Example
-------
>>> from mymodule import make_multisector, CustomSectorMaker1,
    CustomSectorMaker2
>>> port = make_multisector(
>>>     CustomSectorMaker1(),
>>>     CustomSectorMaker2(),
>>>     window_size=7,
>>>     days=250,
>>>     stocks=15,
>>>     price=200,
>>>     weights=[0.2, 0.3, 0.5]
>>> )
File:      ~/indicado